In [ ]:
# Import necessary Python libraries
import io  # Input/output operations
import itertools  # Tools for working with iterators
import math  # Mathematical functions and operations
import os  # Operating system functions
import random  # Generate random numbers and data
import time as time  # Time-related functions and operations

# Specific imports from various libraries
from sklearn.inspection import PartialDependenceDisplay  # Partial dependence display
from gp import Matern  # Import the Matern kernel
from scipy import stats  # Statistics and scientific computing
from autooed.utils.plot import (  # Plotting utility functions
    plot_performance_space,
    plot_performance_metric,
    plot_performance_space_diffcolor,
)
from scipy.interpolate import BSpline, make_interp_spline  # B-spline interpolation
from autooed.utils.pareto import find_pareto_front  # Pareto front finding
from pymoo.performance_indicator.hv import Hypervolume  # Hypervolume calculation
from autooed.utils.pareto import convert_minimization  # Conversion for minimization problems
from scipy.optimize import minimize  # Optimization functions
from scipy import ndimage, misc  # Image processing and manipulation
from scipy.stats import pearsonr  # Pearson correlation
from numpy import cov  # Covariance calculation
from sklearn.decomposition import PCA  # Principal Component Analysis
from sklearn.manifold import TSNE  # T-Distributed Stochastic Neighbor Embedding
from sklearn.utils.optimize import _check_optimize_result  # Utility for checking optimization results
from sklearn.preprocessing import MinMaxScaler  # Min-Max scaling
from sklearn.preprocessing import PolynomialFeatures  # Polynomial features
from sklearn.linear_model import LinearRegression  # Linear regression model
from sklearn.gaussian_process.kernels import Matern as MaternKernel, _check_length_scale  # Matern kernel for Gaussian Processes
from sklearn.gaussian_process.kernels import RBF, ConstantKernel  # Radial Basis Function and Constant Kernel for Gaussian Processes
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel  # Dot Product and White Noise Kernel for Gaussian Processes
from sklearn.gaussian_process import GaussianProcessRegressor  # Gaussian Process Regressor
from sklearn.datasets import make_friedman2  # Generate Friedman-2 dataset
from sklearn.neighbors import KNeighborsRegressor  # K-Nearest Neighbors regressor
from sklearn.neural_network import MLPRegressor  # Multi-layer Perceptron regressor
from sklearn import svm  # Support Vector Machines
from sklearn.tree import export_graphviz  # Export Decision Tree to Graphviz format
from sklearn.metrics import (  # Model evaluation metrics
    r2_score,
    mean_squared_error,
    mean_gamma_deviance,
    mean_absolute_error,
)
from sklearn.model_selection import (  # Model selection and evaluation
    train_test_split,
    GridSearchCV,
    LeaveOneOut,
)
from sklearn.pipeline import make_pipeline  # Construct a pipeline of transformers and estimators
from sklearn.ensemble import RandomForestRegressor  # Random Forest regressor
from sklearn.inspection import partial_dependence  # Partial dependence for scikit-learn models
from sklearn import preprocessing  # Data preprocessing and scaling
from random import randint  # Generate random integers
from random import seed  # Seed for random number generation
from sklearn.tree import DecisionTreeRegressor  # Decision Tree regressor
from sklearn import tree  # Decision Tree for visualization
from sklearn import ensemble  # Ensemble methods
from sklearn import linear_model  # Linear models for regression and classification
import seaborn as sns  # Data visualization library based on Matplotlib
from matplotlib import rc, rcParams  # Matplotlib settings and parameters
import matplotlib as mpl  # Matplotlib for data visualization
import matplotlib.pyplot as plt  # Plotting library
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations and arrays
from scipy.optimize import curve_fit  # Curve fitting
from scipy.stats import beta  # Beta distribution functions
%matplotlib inline  # Display matplotlib plots in Jupyter Notebook


In [ ]:
# Check the font settings
# Switch to Arial in Linux

# If not working, delete ~/.catch/matplotlib
plt.rcParams["font.family"] = "Arial"
plt.rcParams['ps.useafm'] = True
plt.rcParams['pdf.fonttype'] = 42

# Create a FontManager
mpl.font_manager.FontManager()

# Set font weight to bold
rc('font', weight='bold')

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5, 4))

# Plot a scatter plot
plt.scatter([10, 55], [10, 55])

# Customize axis ticks
ax.tick_params(axis='both', length=0, width=1.5, colors='black', grid_alpha=0, labelsize=20)

# Set the x-axis label with a specific font
plt.xlabel('Arial Font Example', fontname='Arial', fontsize=30, fontweight='bold')

# Define custom x-axis ticks
l = np.arange(0, 73, 20)


In [ ]:
# Create a PCA map with Printability as color

# Read and preprocess the evaluated samples data
df_eval = pd.read_csv('./Datasets/evaluated_samples.csv')
df_eval['Toughness(MJ/m3)'] = -df_eval['Toughness(MJ/m3)']
df_eval['Tensile_Strength(MPa)'] = -df_eval['Tensile_Strength(MPa)']

# Read and preprocess the initial data
df_init = pd.read_csv('./Datasets/initial_samples.csv')
df_init['Printability'] = df_init['Printability'].map(dict(Y=1, N=0))

# Select relevant columns for PCA
x_eval = df_eval.iloc[:, 1:6]
x_init = df_init.iloc[:, 1:6]

# Perform PCA dimension reduction
pca = PCA(n_components=2)
x_eval_pca = pca.fit_transform(x_eval)
x_init_pca = pca.fit_transform(x_init)

# Add PCA components to the dataframes
df_eval['pca1'] = x_eval_pca[:, 0]
df_eval['pca2'] = x_eval_pca[:, 1]
df_init['pca1'] = x_init_pca[:, 0]
df_init['pca2'] = x_init_pca[:, 1]

# Add 'initial' labels and concatenate dataframes
df_eval['initial'] = 'Eval.'
df_init['initial'] = 'Initial'
df = pd.concat([df_init, df_eval], axis=0)

# Create a scatter plot
fig, ax = plt.subplots(figsize=(6, 4))
hue = df['initial'].astype(str) + ', ' + df['Printability'].astype(str)
label = ['Init., Printable', 'Init., Non-printable',
         'Eval., Printable', 'Eval., Non-printable']
sns.scatterplot(data=df, x='pca1', y='pca2',
                markers=['.', '.', '*', '*'], edgecolor=None,
                palette=['blue', 'black', 'magenta', 'black'], hue=hue, style=hue,
                alpha=1, s=120)
plt.ylim(-0.7, 0.7)

# Add legend
plt.legend(ncol=2, title='1=Printable; 0=Non-printable', framealpha=1)

# Set axis labels and formatting
ax.set_xlabel('PC1', fontsize=20, fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('PC2', fontsize=20, fontname='Arial', fontweight='bold', labelpad=5)

ax.tick_params(direction='out', length=5, width=3, colors='black',
               grid_alpha=1, labelsize=18)

[i.set_linewidth(3) for i in ax.spines.values()]

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/pca_printable_non.jpeg', dpi=500)


In [ ]:
# Create a PCA map with Tg as color
Tg = df['Tg']
Tg_group = [1 if 9.9 < i < 60.1 else 0 for i in Tg]
df['Tg_group'] = Tg_group

# Create a scatter plot with PCA components and Tg color-coded
fig, ax = plt.subplots(figsize=(6, 4))
hue = df['initial'].astype(str) + ', ' + df['Tg_group'].astype(str)
label = ['Init., Printable', 'Init., Non-printable',
         'Eval., Printable', 'Eval., Non-printable']
sns.scatterplot(data=df, x='pca1', y='pca2',
                markers=['.', '.', '*', '*'], edgecolor=None,
                palette=['blue', 'black', 'magenta', 'black'], hue=hue, style=hue,
                alpha=1, s=120)

# Set y-axis limits
plt.ylim(-0.7, 0.7)

# Add legend
plt.legend(ncol=2, title='1=Des. Tg; 0=Non-des. Tg', framealpha=1)

# Set axis labels and formatting
ax.set_xlabel('PC1', fontsize=20, fontname='Arial',
              fontweight='bold', labelpad=5)
ax.set_ylabel('PC2', fontsize=20, fontname='Arial',
              fontweight='bold', labelpad=5)

# Customize axis tick parameters
ax.tick_params(direction='out', length=5, width=3,
               colors='black', grid_alpha=1, labelsize=18)

# Set linewidth for axis spines
[i.set_linewidth(3) for i in ax.spines.values()]

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/pca_tg_non.jpeg', dpi=500)


In [ ]:
# Separate the dataset to only printable (nonzero) then eval vs. initial
df_nonzero = df.loc[df['Printability'] == 1]
df_nonzero_eval = df_nonzero.loc[df_nonzero['initial'] == 'Eval.']
df_nonzero_init = df_nonzero.loc[df_nonzero['initial'] == 'Initial']
x_eval_nonzero = df_nonzero_eval.iloc[:, 1:6]
x_init_nonzero = df_nonzero_init.iloc[:, 1:6]

X = df_nonzero.iloc[:, 1:6]


In [ ]:
# Plot PCA with Eval. vs. initial label
fig, ax = plt.subplots(figsize=(6, 4))
markers = ['o', 's', '*', 'v']

# Scatter plot for Evaluated samples
sns.scatterplot(data=df_eval, x='pca1', y='pca2',
                markers='*', edgecolor=None, color='blue',
                alpha=0.9, s=50, label='Evaluated')

# Scatter plot for Initial samples with different markers
sns.scatterplot(data=df_init, x='pca1', y='pca2',
                markers=markers, edgecolor=None, color='red',
                alpha=0.9, s=50, label='Initial')

# Set axis labels and formatting
ax.set_xlabel('PC1', fontsize=20, fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('PC2', fontsize=20, fontname='Arial', fontweight='bold', labelpad=5)

# Customize axis tick parameters
ax.tick_params(direction='out', length=5, width=3, colors='black',
               grid_alpha=1, labelsize=18)

# Set linewidth for axis spines
[i.set_linewidth(3) for i in ax.spines.values()]

# Add legend and adjust layout
plt.legend(ncol=2, loc='upper left')
plt.tight_layout()

# Save the figure
plt.savefig('./Figures/eval_init_samples_pca.jpeg', dpi=500)


In [ ]:
# Plot Hypervolume improvement for evaluated samples.
Strength = df_eval['Tensile_Strength(MPa)']
Toughness = df_eval['Toughness(MJ/m3)']
Y_eval = []
for i, j in zip(Strength, Toughness):
    Y_eval.append([i, j])
Y_eval = np.array(Y_eval)

# Negate the values
Y_eval = -Y_eval

# Plot the Hypervolume improvement using a custom function
plot_performance_metric(Y_eval, ['min', 'min'])


In [ ]:
# Calculate the Pareto Front
Strength = df_nonzero['Tensile_Strength(MPa)']
Toughness = df_nonzero['Toughness(MJ/m3)']
Y = []
for i, j in zip(Strength, Toughness):
    Y.append([i, j])
Y = np.array(Y)

# Negate the values
Y = -Y

# Find the Pareto Front
pareto = find_pareto_front(Y)

# Identify the rows corresponding to the Pareto Front in the original DataFrame
pareto_id = []
for i in range(len(df)):
    if df['Tensile_Strength(MPa)'].iloc[i] in -pareto[:, 0]:
        if df['Toughness(MJ/m3)'].iloc[i] in -pareto[:, 1]:
            pareto_id.append(i)

# Create a DataFrame containing the Pareto Front data
df_pareto = df.iloc[pareto_id]

# Access the 'Toughness(MJ/m3)' values in the Pareto Front DataFrame
pareto_toughness = df_pareto['Toughness(MJ/m3)']


In [ ]:
# Define the iterations and the corresponding number of Pareto points
iterations = np.arange(0, 72, 10)
iterations[-1] = 72
pareto_cont = [0, 0, 2, 2, 4, 4, 4, 5]

# Create a scatter plot of the number of Pareto points at each iteration
fig, ax = plt.subplots(figsize=(5, 4))
plt.scatter(iterations, pareto_cont)
plt.plot(iterations, pareto_cont)

# Set axis labels and formatting
ax.set_xlabel('Sample ID', fontsize=20, fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('# of Pareto points', fontsize=20, fontname='Arial', fontweight='bold', labelpad=10)

ax.tick_params(direction='out', length=4, width=2, colors='black', grid_alpha=1, labelsize=15)
[i.set_linewidth(1.5) for i in ax.spines.values()]

# Set the x-axis ticks
plt.xticks(np.arange(0, 73, 12))

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/numParet_iters.png', dpi=500)


In [ ]:
# Create a scatter plot with markers' size assigned to iteration number
fig, ax = plt.subplots(figsize=(5, 4))

# Plot the evaluated samples with red markers and marker size based on 'sample' column
sns.scatterplot(x='Tensile_Strength(MPa)', y='Toughness(MJ/m3)',
                size='sample', color='red', sizes=(1, 200), label='Eval.',
                alpha=0.8, palette=["red"], data=df_eval)

# Plot the initial samples with blue markers
plt.scatter(df_init['Tensile_Strength(MPa)'],
            df_init['Toughness(MJ/m3)'], label='Initial',
            alpha=0.7, color='blue')

# Add a legend
plt.legend()

# Set axis labels and formatting
ax.set_xlabel('$\sigma$$_{T}$ $(MPa)$', fontsize=15, fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('U$_T$ $({MJ}{m^{-3}}$)', fontsize=15, fontname='Arial', fontweight='bold', labelpad=5)

ax.tick_params(direction='out', length=4, width=2, colors='black',
               grid_alpha=1, labelsize=15)

# Set linewidth for axis spines
[i.set_linewidth(1.5) for i in ax.spines.values()]

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/evaluate_init.png', dpi=500)


In [ ]:
# Create a scatter plot with Eval. vs. Initial, same marker size
fig, ax = plt.subplots(figsize=(5, 4))

# Scatter plot for Eval. samples in red
sns.scatterplot(x='Tensile_Strength(MPa)', y='Toughness(MJ/m3)',
                color='red', s=60, label='Eval.',
                alpha=0.6, palette=["blue"], data=df_eval)

# Scatter plot for Initial samples in blue
sns.scatterplot(x='Tensile_Strength(MPa)', y='Toughness(MJ/m3)',
                color='blue', s=60, label='Init.', edgecolor=None,
                alpha=0.6, palette=["red"], data=df_init)
plt.legend()

# Set axis labels and formatting
ax.set_xlabel('σ$_{T}$ (MPa)', fontsize=15,
              fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel(r'U$_T$ (MJ.m$^{-3}$)', fontsize=15,
              fontname='Arial', fontweight='bold', labelpad=5)

# Set tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black',
               grid_alpha=1, labelsize=15)

# Plot Pareto front points
plt.scatter(-pareto[:, 0], -pareto[:, 1], facecolors='none',
            s=60, edgecolors='black', marker='o', alpha=1)

# Sort and interpolate Pareto points
pareto_df = pd.DataFrame({})
pareto_df['strength'] = -pareto[:, 0]
pareto_df['toughness'] = -pareto[:, 1]
pareto_df_sort_str = pareto_df.sort_values(by='strength')[0:5]
new_iters = np.arange(pareto_df_sort_str['strength'].iloc[0],
                      pareto_df_sort_str['strength'].iloc[-1], 0.1)

gfg_str = make_interp_spline(pareto_df_sort_str['strength'],
                             pareto_df_sort_str['toughness'], k=1)
strength_new = gfg_str(new_iters)

# Set linewidth for axis spines
[i.set_linewidth(1.5) for i in ax.spines.values()]

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/pareto_points.png', dpi=500)


In [ ]:
# Create a scatter plot with Eval. vs. Initial, same marker size
fig, ax = plt.subplots(figsize=(5, 4))

# Scatter plot for Eval. samples in red
sns.scatterplot(x='Tensile_Strength(MPa)', y='Toughness(MJ/m3)',
                color='red', s=60, label='Eval.',
                alpha=0.6, palette=["blue"], data=df_eval)

# Scatter plot for Initial samples in blue
sns.scatterplot(x='Tensile_Strength(MPa)', y='Toughness(MJ/m3)',
                color='blue', s=60, label='Init.', edgecolor=None,
                alpha=0.6, palette=["red"], data=df_init)
plt.legend()

# Set axis labels and formatting
ax.set_xlabel('σ$_{T}$ (MPa)', fontsize=15,
              fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel(r'U$_T$ (MJ.m$^{-3}$)', fontsize=15,
              fontname='Arial', fontweight='bold', labelpad=5)

# Set tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black',
               grid_alpha=1, labelsize=15)

# Plot Pareto front points and connect them
plt.scatter(-pareto[:, 0], -pareto[:, 1], facecolors='none',
            s=60, edgecolors='black', marker='o', alpha=1,)

# Set legend
plt.legend()

# Interpolate and plot the Pareto front curve
new_iters = np.arange(pareto_df_sort_str['strength'].iloc[0],
                      pareto_df_sort_str['strength'].iloc[-1], 0.1)

gfg_str = make_interp_spline(pareto_df_sort_str['strength'],
                             pareto_df_sort_str['toughness'], k=1)
strength_new = gfg_str(new_iters)
# Connect the points
plt.plot(new_iters, strength_new, 'k')

# Set linewidth for axis spines
[i.set_linewidth(1.5) for i in ax.spines.values()]

# Save the figure
plt.tight_layout()
plt.savefig('./Figures/paretofront_area.png', dpi=500)


In [ ]:
# Calculate max. of hypervolume indicator
def calc_hypervolume(Y_eval, ref_point, obj_type=None):
    '''
    Calculate hypervolume
    '''
    # Convert Y_eval to minimization if needed
    # Y_eval = convert_minimization(Y_eval, obj_type)

    # Calculate the hypervolume
    return Hypervolume(ref_point=ref_point).calc(Y_eval)

# Define the reference point for hypervolume calculation
ref_point = (0, 0)

# Calculate the hypervolume for Y_eval
hypervolume = calc_hypervolume(Y_eval, ref_point)

# Print the hypervolume
print(hypervolume)


In [ ]:
# Define objectives variables

# Initial points
Strength_init = df_init['Tensile_Strength(MPa)'].loc[df_init['Printability'] == 1]
Toughness_init = df_init['Toughness(MJ/m3)'].loc[df_init['Printability'] == 1]
Y_initial = []

for i, j in zip(Strength_init, Toughness_init):
    Y_initial.append([i, j])

Y_initial = np.array(Y_initial)

# Evaluated points
Strength_eval = df_eval['Tensile_Strength(MPa)'].loc[df_eval['Printability'] == 1]
Toughness_eval = df_eval['Toughness(MJ/m3)'].loc[df_eval['Printability'] == 1]
Y_eval = []

for i, j in zip(Strength_eval, Toughness_eval):
    Y_eval.append([i, j])
    
Y_eval = np.array(Y_eval)


In [ ]:
# Define the Kernel functions for GPs
n_var = 5
nu1 = 1
nu2 = 1

main_kernel1 = Matern(length_scale=1 * np.ones(n_var),
                      length_scale_bounds=(np.sqrt(13e-1), np.sqrt(5e5)), nu=0.5 * nu1)

main_kernel2 = Matern(length_scale=2 * np.ones(n_var),
                      length_scale_bounds=(np.sqrt(1.65e-1), np.sqrt(1e3)), nu=0.5 * nu2)

kernel1 = ConstantKernel(constant_value=1.0,
                         constant_value_bounds=(np.sqrt(1e-5), np.sqrt(1000))) * main_kernel1 + \
    ConstantKernel(constant_value=1.0,
                   constant_value_bounds=(np.exp(-100), np.exp(0)))

kernel2 = ConstantKernel(constant_value=1.0,
                         constant_value_bounds=(np.sqrt(0.01), np.sqrt(35))) * main_kernel2

kernel2 = ConstantKernel(constant_value=1.0,
                         constant_value_bounds=(np.sqrt(1e-5), np.sqrt(100))) * main_kernel2 + \
    ConstantKernel(constant_value=1.0,
                   constant_value_bounds=(np.exp(-10), np.exp(0)))



In [ ]:
# Define the GPs

def constrained_optimization(obj_func, initial_theta, bounds):
    '''
    Customized version of constrained optimization to avoid convergence warning.
    '''
    opt_res = minimize(obj_func, initial_theta,
                       method="L-BFGS-B", jac=True, bounds=bounds)
    # NOTE: Temporarily disable the checking below because of the numerical instability sometimes.
    _check_optimize_result("lbfgs", opt_res)
    return opt_res.x, opt_res.fun

gpr_strength = GaussianProcessRegressor(
    kernel=kernel1,
    optimizer=constrained_optimization,
    alpha=0.2
)

gpr_toughness = GaussianProcessRegressor(
    kernel=kernel1,
    optimizer=constrained_optimization,
    alpha=0.3
)


In [ ]:
# Train Gaussian Process models iteratively and predict the next point

# Initialize lists to store errors and predictions
errors_strength0 = []
pred_strength = []
sdt_strength = []
i = 0
X = np.array(X)
Y = -np.array(Y)

# GP for Strength
for i in range(0, len(Y_eval)):
    gpr_strength = gpr_strength.fit(
        X[0:len(Y_initial)+i, :], Y[0:len(Y_initial)+i, 0])
    x_next = np.array(X[len(Y_initial) + i, :])
    x_next = x_next.reshape(1, 5)
    y_next = np.array(Y[len(Y_initial) + i, 0])
    y_next = y_next.reshape(1, 1)
    try:
        y_nexts = np.array(Y[len(Y_initial) + i: len(Y_initial) + i+5, 0])
        y_nexts = y_nexts.reshape(5, 1)
        x_nexts = np.array(X[len(Y_initial) + i: len(Y_initial) + i+5, :])
        x_nexts = x_nexts.reshape(5, 5)
        score_strength = gpr_strength.score(x_nexts, y_nexts)
        #print (score_strength)
    except:
        pass

    gpr_strength_predict = gpr_strength.predict(x_next, return_std=True)
    pred_strength.append([gpr_strength_predict[0][0], y_next[0][0]])
    sdt_strength.append(gpr_strength_predict[1][0])

    #print (gpr_strength_predict, y_next)

    error = abs(gpr_strength_predict[0] - y_next)/y_next

    #print (error)
    errors_strength0.append(error)

# GP for Toughness
errors_toughness0 = []
pred_toughness = []
sdt_toughness = []
for i in range(0, len(Y_eval)):
    gpr_toughness = gpr_toughness.fit(
        X[0:len(Y_initial)+i, :], Y[0:len(Y_initial)+i, 1])

    x_next = np.array(X[len(Y_initial) + i, :])
    x_next = x_next.reshape(1, 5)
    y_next = np.array(Y[len(Y_initial) + i, 1])
    y_next = y_next.reshape(1, 1)
    try:
        y_nexts = np.array(Y[len(Y_initial)+i: len(Y_initial)+i+5, 1])
        y_nexts = y_nexts.reshape(5, 1)
        x_nexts = np.array(X[len(Y_initial)+i: len(Y_initial)+i+5, :])
        x_nexts = x_nexts.reshape(5, 5)
        score_toughness = gpr_toughness.score(x_nexts, y_nexts)
        #print (score_toughness)
    except:
        pass

    gpr_toughness_predict = gpr_toughness.predict(x_next, return_std=True)
    pred_toughness.append([gpr_toughness_predict[0][0], y_next[0][0]])
    sdt_toughness.append(gpr_toughness_predict[1][0])

    # Avoid division by zero
    error = abs(gpr_toughness_predict[0] - y_next)/(y_next+1)
    errors_toughness0.append(error)



In [ ]:
# Plot predictions of GPs after trained on all samples vs. True experimental values

fig, ax = plt.subplots(figsize=(5, 4))

# Set the range of data to visualize
start_index = 0
end_index = 100

# Plot predicted vs. true values for Strength
pred_strength = np.array(pred_strength)
plt.scatter(pred_strength[start_index:end_index, 1],
            pred_strength[start_index:end_index, 0])
plt.xlabel('True Strength')
plt.ylabel('Predicted Strength')
plt.plot([0, 60], [0, 60], '--r')

# Calculate correlation and R-squared for Strength
print(pearsonr(pred_strength[start_index:end_index, 0], pred_strength[start_index:end_index, 1]))
print(r2_score(pred_strength[start_index:end_index, 1], pred_strength[start_index:end_index, 0]))

# Create a new figure for Toughness
fig, ax = plt.subplots(figsize=(5, 4))

# Plot predicted vs. true values for Toughness
pred_toughness = np.array(pred_toughness)
plt.scatter(pred_toughness[start_index:end_index, 1],
            pred_toughness[start_index:end_index, 0])
plt.plot([0, 30], [0, 30], '--r')
plt.xlabel('True Toughness')
plt.ylabel('Predicted Toughness')

# Calculate correlation and R-squared for Toughness
print(pearsonr(pred_toughness[start_index:end_index, 0], pred_toughness[start_index:end_index, 1]))
print(r2_score(pred_toughness[start_index:end_index, 1], pred_toughness[start_index:end_index, 0]))

In [ ]:
# Plot errors of predictions over iterations for Strength

# Create copies of the error lists
errors_strength_ = errors_strength0.copy()
errors_toughness_ = errors_toughness0.copy()

# Reshape error arrays
errors_strength = np.array(errors_strength0).reshape(len(Y_eval), 1)
errors_toughness = np.array(errors_toughness0).reshape(len(Y_eval), 1)

# Define the interval for averaging errors
n = 5

# Define iterations
iters = list(np.arange(n - 1, len(Y_eval) + 4, n))
x_iter = iters

# Calculate average errors over every n evaluations
avgerrors_strength = np.average(errors_strength.reshape(-1, n), axis=1)
avgerrors_toughness = np.average(errors_toughness.reshape(-1, n), axis=1)

# Create new iteration values
new_iters = np.arange(48 - 43, 112.1 - 43, 0.1)

# Create interpolation functions
gfg_str = make_interp_spline(x_iter, avgerrors_strength, bc_type='clamped')
gfg_tough = make_interp_spline(x_iter, avgerrors_toughness, bc_type='clamped')

# Calculate smoothed error values
strength_new = gfg_str(new_iters)
toughness_new = gfg_tough(new_iters)

# Create a new figure for plotting
fig, ax = plt.subplots(figsize=(5, 4))

# Scatter plot of average errors for Strength
plt.scatter(x_iter, avgerrors_strength, s=30, alpha=1, color='red')

# Calculate a trendline for Strength
x = np.arange(0, len(Y_eval), n) + n + 43
x = list(x)
y = avgerrors_strength
z = np.polyfit(x, y, 1)
p = np.poly1d(z)

# Add the Strength trendline to the plot
plt.plot(x_iter, p(x), '--k')

# Add smoothed Strength error values to the plot
plt.plot(new_iters, strength_new, color='red', label='$\sigma$$_{T}$', alpha=1)

# Set plot labels and legend
plt.legend(['$\sigma$$_{T}$', 'Trend'], fontsize=13)
ax.set_xlabel('Sample ID', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Average RE', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)

# Set plot parameters
ax.tick_params(direction='out', length=3.5, width=3, colors='black', grid_alpha=1, labelsize='18')
[i.set_linewidth(2) for i in ax.spines.values()]
plt.ylim((0, 1))
plt.tight_layout()
plt.savefig('./Figures/AverageRE_strength.png', dpi=500)


In [ ]:
# Plot errors of predictions over iterations for Toughness
errors_strength_ = errors_strength0.copy()
errors_toughness_ = errors_toughness0.copy()

# Reshape error arrays to have the desired dimensions
errors_strength = np.array(errors_strength0).reshape(len(Y_eval), 1)
errors_toughness = np.array(errors_toughness0).reshape(len(Y_eval), 1)

# Calculate the average error over every n evaluations
n = 5
iters = list(np.arange(n - 1, len(Y_eval) + 4, n))
x_iter = iters
avgerrors_strength = np.average(errors_strength.reshape(-1, n), axis=1)

# Comment: The following line seems to be commented out. 
# avgerrors_strength[8] = 0.4

avgerrors_toughness = np.average(errors_toughness.reshape(-1, n), axis=1)

# Define a new set of iterations
new_iters = np.arange(4, 112.1 - 43, 0.1)

# Create interpolation functions for error values
gfg_str = make_interp_spline(x_iter, avgerrors_strength, bc_type='natural')
gfg_tough = make_interp_spline(x_iter, avgerrors_toughness, k=3, bc_type='clamped')

# Generate smoothed error values using the interpolation functions
strength_new = gfg_str(new_iters)
toughness_new = gfg_tough(new_iters)

# Create a plot
fig, ax = plt.subplots(figsize=(5, 4))

# Scatter plot of average toughness errors
plt.scatter(x_iter, avgerrors_toughness, s=30, alpha=0.7, color='blue')

# Calculate a trendline for toughness errors
x = (np.arange(0, len(Y_eval), n) + n + 39)
x = list(x)
y = avgerrors_toughness
z = np.polyfit(x, y, 1)
p = np.poly1d(z)

# Add the trendline to the plot
plt.plot(x_iter, p(x), '--k')

# Plot the smoothed toughness errors
plt.plot(new_iters, toughness_new, color='blue', label='Toughness', alpha=0.7)
plt.legend(['U$_T$', 'Trend'], fontsize=13)

# Set labels and formatting for the plot
ax.set_xlabel('Sample ID', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Average RE', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)

# Adjust plot settings
ax.tick_params(direction='out', length=3.5, width=3, colors='black', grid_alpha=1, labelsize='18')
[i.set_linewidth(2) for i in ax.spines.values()]
plt.ylim((0, 2.5))

# Save the plot as an image
plt.tight_layout()
plt.savefig('./Figures/AverageRE_toughness.png', dpi=500)


In [ ]:
# Plot sum of hard monomers over iteration

# Extract the hard monomer counts from the input data
x_eval = np.array(x_eval_nonzero)

# Define the number of evaluations to consider for averaging
n = 5

# Create a list of iterations based on the specified interval
iters = list(np.arange(n - 1, len(Y_eval) + 4, n))
x_iter = iters

# Calculate the average hard monomer count for each set of n evaluations
x_eval_hard = x_eval[:, 2].ravel() + x_eval[:, 3].ravel() + x_eval[:, 4].ravel() + \
    np.round(1 - (x_eval[:, 2].ravel() + x_eval[:, 3].ravel() + x_eval[:, 4].ravel() + x_eval[:, 0].ravel() + x_eval[:, 1].ravel()), 2)
avg_x_eval_hard = np.average(x_eval_hard.reshape(-1, n), axis=1)

# Create a scatter plot of the average hard monomer count
fig, ax = plt.subplots(figsize=(5, 4))
plt.scatter(x_iter, avg_x_eval_hard, s=30, alpha=0.7, color='black')

# Define a new set of iterations for smoothing
new_iters = np.arange(4, 69, 0.1)

# Create an interpolation function for the average hard monomer count
gfg_str = make_interp_spline(x_iter, avg_x_eval_hard, k=3, bc_type='natural', check_finite=False)
strength_new = gfg_str(new_iters)

# Plot the mean and smoothed hard monomer count
plt.plot([0, 70], [np.mean(x_eval_hard), np.mean(x_eval_hard)], '--k', label='mean')
plt.legend()
plt.plot(new_iters, strength_new, color='blue', label='Strength', alpha=0.7)

# Set labels and formatting for the plot
plt.xlabel('Sample ID')
plt.ylabel('Ratio of hard monomers')
ax.set_xlabel('Sample ID', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Ratio of hard monomers', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.tick_params(direction='out', length=3.5, width=3, colors='black', grid_alpha=1, labelsize='18')
[i.set_linewidth(2) for i in ax.spines.values()]

# Adjust plot layout and save the figure
plt.tight_layout()
plt.savefig('./Figures/hardmonomers_iter.png', dpi=500)


In [ ]:
# Plot sum of ratios of soft monomers over iteration

# Define the number of evaluations to consider for averaging
n = 5

# Create a list of iterations based on the specified interval
iters = list(np.arange(n - 1, len(Y_eval) + 4, n))
x_iter = iters

# Calculate the average ratio of soft monomers for each set of n evaluations
x_eval_soft = x_eval[:, 0].ravel() + x_eval[:, 1].ravel()
avg_x_eval_soft = np.average(x_eval_soft.reshape(-1, n), axis=1)

# Create a scatter plot of the average ratio of soft monomers
fig, ax = plt.subplots(figsize=(5, 4))
plt.scatter(x_iter, avg_x_eval_soft, s=30, alpha=0.7, color='blue')

# Define a new set of iterations for smoothing
new_iters = np.arange(4, 69, 0.1)

# Create an interpolation function for the average ratio of soft monomers
gfg_str = make_interp_spline(x_iter, avg_x_eval_soft, k=3, bc_type='natural', check_finite=False)
strength_new = gfg_str(new_iters)

# Plot the mean and smoothed ratio of soft monomers
plt.plot([0, 70], [np.mean(x_eval_soft), np.mean(x_eval_soft)], '--k', label='mean')
plt.legend()
plt.plot(new_iters, strength_new, color='red', label='Strength', alpha=0.7)

# Set labels and formatting for the plot
plt.xlabel('Sample ID')
plt.ylabel('Ratio of soft monomers')
ax.set_xlabel('Sample ID', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel('Ratio of soft monomers', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)
ax.tick_params(direction='out', length=3.5, width=3, colors='black', grid_alpha=1, labelsize='18')
[i.set_linewidth(2) for i in ax.spines.values()]

# Adjust plot layout
plt.tight_layout()
plt.savefig('./Figures/softmonomers_iter.png', dpi=500)


In [ ]:
# Retrain the GPs on all evaluated and initial samples
gpr_strength.fit(X, Y[:, 0],);
gpr_toughness.fit(X, Y[:, 1]);

In [ ]:
# Scatter plot of ratios 2by2

# Define the number of dimensions
n_dims = 5

# Set the step size for histogram bins
step = 0.055

# Create a subplot with specified dimensions and adjust spacing
fig, ax = plt.subplots(n_dims, n_dims, figsize=(3 * n_dims, 3 * n_dims))
fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.02, wspace=0.02)

# Define the ratios to be plotted
Ratios = ['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']

for i in range(n_dims):
    for j in range(n_dims):
        if i == j:
            ax_ = ax[i, j]
            if i == 0:
                ax_.xaxis.set_label_position("top")
                ax_.xaxis.tick_top()
                ax_.set_xlabel(Ratios[j], fontweight='bold', fontsize=12)
                ax_.set_ylabel('Rel. abundance', fontweight='bold', fontsize=12)
                min = np.min(df_eval[Ratios[j]])
                max = np.max(df_eval[Ratios[j]])
                ax_.hist(Ratios[j], data=df_eval, color='blue', density=True, bins=np.arange(min, max + 0.1, step=step), alpha=0.4)
                min = np.min(df_init[Ratios[j]])
                max = np.max(df_init[Ratios[j]])
                ax_.hist(Ratios[j], data=df_init, bins=np.arange(min, max + 0.1, step=step), density=True, color='red', alpha=0.4)
            else:
                ax_ = ax[i, j]
                min = np.min(df_eval[Ratios[j]])
                max = np.max(df_eval[Ratios[j]])
                ax_.hist(Ratios[j], data=df_eval, color='blue', density=True, bins=np.arange(min, max + 0.1, step=step), orientation='horizontal', alpha=0.4)
                min = np.min(df_init[Ratios[j]])
                max = np.max(df_init[Ratios[j]])
                ax_.hist(Ratios[j], data=df_init, bins=np.arange(min, max + 0.1, step=step), density=True, orientation='horizontal', color='red', alpha=0.4)
                if i != 0:
                    ax_.xaxis.set_label_position("top")
                    ax_.xaxis.tick_top()
                    ax_.set_xlabel('Rel. abundance', fontweight='bold', fontsize=12)
                    ax_.set_yticklabels([])
                    ax_.tick_params('y', length=0, width=0)
                else:
                    ax_.set_ylabel(Ratios[j], fontweight='bold', fontsize=12)
                    ax_.xaxis.set_label_position("top")
                    ax_.xaxis.tick_top()
                    ax_.set_xlabel('Rel. abundance', fontweight='bold', fontsize=12)
        elif i > j:
            ax_ = ax[i, j]
            ax_.scatter(Ratios[j], Ratios[i], c='red', data=df_init, s=40)
            ax_.scatter(Ratios[j], Ratios[i], c='sample', data=df_eval, s=40, cmap='Blues')
            if j == 0:
                ax_.set_ylabel('{}'.format(Ratios[i]), fontweight='bold', fontsize=12)
                if i != (n_dims - 1):
                    ax_.set_xticklabels([])
                    ax_.tick_params('x', length=0, width=0)
            if i == (n_dims - 1):
                ax_.set_xlabel('{}'.format(Ratios[j]), fontweight='bold', fontsize=12)
            if i != (n_dims - 1):
                if j != 0:
                    ax_.tick_params('both', length=0, width=0)
                    ax_.set_xticklabels([])
                    ax_.set_yticklabels([])
            if i == (n_dims - 1):
                if j != 0:
                    ax_.tick_params('y', length=0, width=0)
                    ax_.set_yticklabels([])
            ax_.scatter(Ratios[j], Ratios[i], data=df_pareto, color='yellow', edgecolors='black', s=200, marker='*')
        if j > i:
            ax_ = ax[i, j]
            ax_.axis("off")

# Save the plot as an image
plt.savefig('./Figures/hist_all_input.png', dpi=500)


In [ ]:
# Partial dependence plots of Toughness predicted by GP trained on all samples.

n_dims = 5
step = 0.055
min_z, max_z = -0.2, 10
X_ratios = df_nonzero.iloc[:, 1:6]

# Create a subplot with specified dimensions and adjust spacing
fig, ax = plt.subplots(n_dims, n_dims,
                       figsize=(3 * n_dims, 3 * n_dims))
fig.subplots_adjust(left=0.06, right=0.95, bottom=0.06, top=0.95,
                    hspace=0.032, wspace=0.032)

# Define the ratios to be plotted
Ratios = ['R1(HA)',	'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']
color = ['blue' if i > 1 else 'red']

for i in range(n_dims):
    for j in range(n_dims):
        color = 'red' if i > 1 else 'blue'
        if i == j:
            ax_ = ax[i, j]
            ax_.set_yticks((2, 4, 6, 8, 10))
            
            pd_results = partial_dependence(gpr_toughness, X_ratios, features=[i],
                                            grid_resolution=100, percentiles=(0.05, 0.095))
            ratio = np.array(pd_results["values"])[0]
            part_dep = np.array(pd_results["average"])[0]

            ax_.plot(ratio, part_dep, color=color, markersize=20, linewidth=2)
            ax_.scatter(ratio, part_dep, color='black', s=10,)

            ax_.xaxis.set_label_position("top")
            ax_.xaxis.tick_top()
            ax_.set_xlabel('{}'.format(
                Ratios[i]), fontweight='bold', fontsize=20, labelpad=5)
            ax_.yaxis.set_label_position("right")
            ax_.yaxis.tick_right()
            ax_.set_ylabel('PD (U$_T$)',
                           fontweight='bold', fontsize=20, fontname='Arial', )
            ax_.tick_params('y', length=0, width=0, )
            ax_.set_ylim(min_z, max_z)
            ax_.tick_params(direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20')

        elif i > j:
            ax_ = ax[i, j]
            features = [j, i]
            pd_inter_results = partial_dependence(gpr_toughness, X_ratios,
                                                  features=features, method='brute',
                                                  kind='average', grid_resolution=100, percentiles=(0.05, 0.095))
            XX, YY = np.meshgrid(
                pd_inter_results['values'][0], pd_inter_results['values'][1])
            Z = pd_inter_results['average'][0].T
            Z_level = np.linspace(min_z, max_z, num=8)
            CS = ax_.contour(XX, YY, Z, levels=Z_level,
                             linewidths=0.5, colors="k")
            ax_.contourf(
                XX,
                YY,
                Z,
                levels=Z_level,
                vmin=Z_level[0],
                vmax=Z_level[-1],
            )
            if i == 1 and j == 0:
                ax_.set_yticks((0.1, 0.2, 0.4))
            ax_.tick_params(direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20')

            ax_.tick_params('x', direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20', rotation=90)
            ax_.clabel(CS, fmt="%2.2f", colors="k", fontsize=10, inline=True)
            ax_.scatter(Ratios[j], Ratios[i], data=df_pareto,
                        color='yellow', edgecolors='black',
                        s=200, marker='*')

            if j == 0:
                ax_.set_ylabel('{}'.format(Ratios[i]), fontweight='bold',
                               fontsize=20, fontname='Arial', labelpad=5)
                if i != (n_dims-1):
                    ax_.set_xticklabels([])
                    ax_.tick_params('x', length=0, width=0, )
            if i == (n_dims-1):
                ax_.set_xlabel('{}'.format(Ratios[j]), fontweight='bold',
                               fontsize=20, fontname='Arial')

            if i != (n_dims-1):
                if j != 0:
                    ax_.tick_params('both', length=0, width=0, )
                    ax_.set_xticklabels([])
                    ax_.set_yticklabels([])
            if i == (n_dims-1):
                if j != 0:
                    ax_.tick_params('y', length=0, width=0, )
                    ax_.set_yticklabels([])
        if j > i:
            ax_ = ax[i, j]
            ax_.axis("off")
plt.savefig('./Figures/pd_tough', dpi=500)


In [ ]:
# Partial dependence plots of Strength predicted by GP trained on all samples.

n_dims = 5
step = 0.055
min_z, max_z = 10, 32
X_ratios = df_nonzero.iloc[:, 1:6]

fig, ax = plt.subplots(n_dims, n_dims,
                       figsize=(3 * n_dims, 3 * n_dims))
fig.subplots_adjust(left=0.06, right=0.95, bottom=0.06, top=0.95,
                    hspace=0.032, wspace=0.032)
Ratios = ['R1(HA)',	'R2(IA)', 'R3(NVP)',
          'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']
color = ['blue' if i > 1 else 'red']
for i in range(n_dims):
    for j in range(n_dims):
        color = 'red' if i > 1 else 'blue'
        if i == j:
            ax_ = ax[i, j]
            ax_.set_yticks((15, 20, 25, 30))
            pd_results = partial_dependence(gpr_strength, X_ratios, features=[i],
                                            grid_resolution=100, percentiles=(0.05, 0.095))
            ratio = np.array(pd_results["values"])[0]
            part_dep = np.array(pd_results["average"])[0]

            ax_.plot(ratio, part_dep, color=color, markersize=20, linewidth=2)
            ax_.scatter(ratio, part_dep, color='black', s=10,)

            ax_.xaxis.set_label_position("top")
            ax_.xaxis.tick_top()
            ax_.set_xlabel('{}'.format(
                Ratios[i]), fontweight='bold', fontsize=20, labelpad=5)
            ax_.yaxis.set_label_position("right")
            ax_.yaxis.tick_right()
            ax_.set_ylabel('PD (σ$_{T}$)',
                           fontweight='bold', fontsize=20, fontname='Arial', )
            ax_.tick_params('y', length=0, width=0, )
            ax_.set_ylim(min_z, max_z)
            ax_.tick_params(direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20')

        elif i > j:
            ax_ = ax[i, j]
            features = [j, i]

            pd_inter_results = partial_dependence(gpr_strength, X_ratios,
                                                  features=features, method='brute',
                                                  kind='average', grid_resolution=100, percentiles=(0.05, 0.095))

            XX, YY = np.meshgrid(
                pd_inter_results['values'][0], pd_inter_results['values'][1])
            Z = pd_inter_results['average'][0].T
            Z_level = np.linspace(min_z, max_z, num=8)
            CS = ax_.contour(XX, YY, Z, levels=Z_level,
                             linewidths=0.5, colors="k")
            ax_.contourf(
                XX,
                YY,
                Z,
                levels=Z_level,
                vmin=Z_level[0],
                vmax=Z_level[-1],
            )
            if i == 1 and j == 0:
                ax_.set_yticks((0.1, 0.2, 0.4))
            ax_.tick_params(direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20')

            ax_.tick_params('x', direction='out', length=5, width=3, colors='black',
                            grid_alpha=1, labelsize='20', rotation=90)
            ax_.clabel(CS, fmt="%2.2f", colors="k", fontsize=10, inline=True)
            ax_.scatter(Ratios[j], Ratios[i], data=df_pareto,
                        color='yellow', edgecolors='black',
                        s=200, marker='*')

            if j == 0:
                ax_.set_ylabel('{}'.format(Ratios[i]), fontweight='bold',
                               fontsize=20, fontname='Arial', labelpad=5)
                if i != (n_dims-1):
                    ax_.set_xticklabels([])
                    ax_.tick_params('x', length=0, width=0, )
            if i == (n_dims-1):
                ax_.set_xlabel('{}'.format(Ratios[j]), fontweight='bold',
                               fontsize=20, fontname='Arial')

            if i != (n_dims-1):
                if j != 0:
                    ax_.tick_params('both', length=0, width=0, )
                    ax_.set_xticklabels([])
                    ax_.set_yticklabels([])
            if i == (n_dims-1):
                if j != 0:
                    ax_.tick_params('y', length=0, width=0, )
                    ax_.set_yticklabels([])
        if j > i:
            ax_ = ax[i, j]
            ax_.axis("off")
plt.savefig('./Figures/pd_str', dpi=500)


In [ ]:
# Plot Toughness for two ratios at a time. 

X_ratios = df.iloc[:, 1:6]
for soft, hard in itertools.product(range(0, 2), range(2, 5)):
    fig, ax = plt.subplots(1, 3, figsize=(8, 4), )  # constrained_layout=True)

    min_z, max_z = -0.2, 10
    # R1
    ratio_count = soft
    pd_results = partial_dependence(gpr_toughness, X_ratios, features=[ratio_count],
                                    grid_resolution=100, percentiles=(0.05, 0.095))
    ratio = np.array(pd_results["values"])[0]
    part_dep = np.array(pd_results["average"])[0]

    ax_ = ax[0]
    ax_.plot(ratio, part_dep, color='blue', markersize=20, linewidth=2)
    ax_.scatter(ratio, part_dep, color='black', s=10,)
    ax_.tick_params(direction='out', length=4, width=2, colors='black',
                    grid_alpha=1, labelsize='15')
    ax_.set_xlabel('{}'.format(Ratios[ratio_count]), fontweight='bold',
                   fontsize=15, fontname='Arial')
    ax_.set_ylabel('Partial dependence', fontsize=15,
                   fontname='Arial', fontweight='bold',)
    ax_.set_ylim(min_z, max_z)
    ratio1 = Ratios[ratio_count]
    # R4
    ratio_count = hard
    pd_results = partial_dependence(gpr_toughness, X_ratios, features=[ratio_count],
                                    grid_resolution=100, percentiles=(0, 1))
    ratio = np.array(pd_results["values"])[0]
    part_dep = np.array(pd_results["average"])[0]

    ax_ = ax[1]
    ax_.plot(ratio, part_dep, color='red', markersize=15)
    ax_.scatter(ratio, part_dep, color='black', s=10,)
    ax_.set_yticklabels([])
    ax_.tick_params('x', direction='out', length=4, width=2, colors='black',
                    grid_alpha=1, labelsize='15')
    ax_.tick_params('y', direction='out', length=0, width=0, colors='black',
                    grid_alpha=1, labelsize='15')

    ax_.set_xlabel('{}'.format(Ratios[ratio_count]), fontweight='bold',
                   fontsize=15, fontname='Arial')
    ax_.set_ylabel('', fontsize=15,
                   fontname='Arial', fontweight='bold',)
    ax_.set_ylim(min_z, max_z)
    ratio2 = Ratios[ratio_count]

    # Interaction of R1 and R4
    ax_ = ax[2]
    ratio_count = 0
    X_ratios = df.iloc[:, 1: 6]
    features = [soft, hard]
    pd_inter_results = partial_dependence(gpr_toughness, X_ratios, features=features, method='brute',
                                          kind='average',
                                          grid_resolution=100, percentiles=(0.05, 0.095))

    XX, YY = np.meshgrid(
        pd_inter_results['values'][0], pd_inter_results['values'][1])
    Z = pd_inter_results['average'][0].T
    Z_level = np.linspace(min_z, max_z, num=8)
    CS = ax_.contour(XX, YY, Z, levels=Z_level, linewidths=0.5, colors="k")

    ax_.contourf(
        XX,
        YY,
        Z,
        levels=Z_level,
        vmin=Z_level[0],
        vmax=Z_level[-1],
    )
    ax_.yaxis.set_label_position("right")
    ax_.yaxis.tick_right()

    ax_.tick_params(direction='out', length=4, width=2, colors='black',
                    grid_alpha=1, labelsize='15')
    ax_.set_xlabel('{}'.format(Ratios[soft]), fontweight='bold',
                   fontsize=15, fontname='Arial')
    ax_.set_ylabel('{}'.format(Ratios[hard]), fontweight='bold',
                   fontsize=15, fontname='Arial')
    ax_.clabel(CS, fmt="%2.2f", colors="k", fontsize=10, inline=True)
    # save the figure
    fig.subplots_adjust(left=0.08, right=0.9, bottom=0.138, top=0.95,
                        hspace=0.02, wspace=0.03)
    # plt.tight_layout()
    plt.savefig('./Figures/pd_tough_{}_{}'.format(ratio1, ratio2), dpi=500)


In [ ]:
# Simulatoin, not real data, hypervolume indicator
Xs = [3, 6, 7, 9, 2]
Ys = [16, 10, 7, 4, 2.5]

# Create a plot
fig, ax = plt.subplots(figsize=(5, 4))

# Scatter plot the data points
plt.scatter(Xs, Ys, color='black')

# Labels for the data points
labels = ['a', 'b', 'c', 'd', 'r']
shiftx = [-0.1, +0.2, 0.2, -0.1, -0.4]
shifty = [0.5, -1, 0., 0.5, -0.5]

# Annotate the data points with labels
for i, txt in enumerate(labels):
    ax.annotate(txt, (Xs[i] + shiftx[i], Ys[i] + shifty[i]))

# Set the plot limits
plt.xlim(0, 12)
plt.ylim(0, 22)

# Customize the plot's visual elements
[i.set_linewidth(1) for i in ax.spines.values()]

XCs = [2, 5.5, 6.5, 5, 8.5, 10]
YCs = [17, 14.5, 16, 7.5, 5.5, 2.5]

# Adjust labels for the second set of data points
shiftx = [-0.1, +0.2, 0.2, 0.2, -0.1, -0.1]
shifty = [0.5, -1, 0., 0., 0.45, 0.5]

# Scatter plot the second set of data points
plt.scatter(XCs, YCs, color='white', edgecolors='black')

# Labels for the second set of data points
labels = [r'C$_1$', r'C$_3$', r'C$_2$', r'C$_4$', r'C$_5$', r'C$_6$']
for i, txt in enumerate(labels):
    ax.annotate(txt, (XCs[i] + shiftx[i], YCs[i] + shifty[i]), fontsize=10)

# Set labels for the x and y axes
ax.set_xlabel(r'$f_1$', style='italic', fontsize=15, fontname='Arial', fontweight='bold', labelpad=5)
ax.set_ylabel(r'$f_2$', style='italic', fontsize=15, fontname='Arial', fontweight='bold', labelpad=5)

# Customize tick parameters
ax.tick_params(axis='both', length=3, width=1.5, colors='black', grid_alpha=0, labelsize=10)

# Save the plot to an image file
plt.tight_layout()
plt.savefig('./Figures/EHVI_simulated_graph.png', dpi=500)


In [ ]:
# Plot Tg vs. Strength, comparing Initial vs. Evaluated.

# Set the 'Sample' column in the DataFrame to 'initial'
df_nonzero['Sample'] = df_nonzero['initial']

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(5, 4))

# Create a scatterplot to compare Tg and Tensile Strength, differentiating by 'Sample'
sns.scatterplot(data=df_nonzero, x='Tg', y='Tensile_Strength(MPa)', palette=['blue', 'red'],
                hue='Sample', edgecolor='black')

# Set the y-axis label and properties
ax.set_ylabel('σ$_{T}$ (MPa)', fontsize=18, fontname='Arial', fontweight='bold', labelpad=5)

# Set the x-axis label and properties
ax.set_xlabel(r'$T_g$ ( °C)', fontsize=18, fontname='Arial', fontweight='bold', labelpad=5)

# Customize tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black', grid_alpha=1, labelsize=15)

# Add vertical dashed lines at x=10 and x=60
plt.axvline(x=10, linestyle="--", color='k')
plt.axvline(x=60, linestyle="--", color='k')

# Ensure a tight layout for the plot
plt.tight_layout()

# Save the plot to an image file
plt.savefig('./Figures/Tg_Strength.png', dpi=500)


In [ ]:
# Plot Tg vs. Strength, only Initial

fig, ax = plt.subplots(figsize=(5, 4))
sns.scatterplot(data=df_nonzero.loc[df_nonzero['Sample']=='Initial'], x='Tg', y='Tensile_Strength(MPa)', palette=['blue'],
                hue='Sample',   edgecolor='black', )

ax.set_ylabel('σ$_{T}$ (MPa)', fontsize='18',
              fontname='Arial', fontweight='bold', labelpad=5)
ax.set_xlabel(r'$T_g$ ( °C)', fontsize='18',
              fontname='Arial', fontweight='bold', labelpad=5)
ax.tick_params(direction='out', length=4, width=2, colors='black',
               grid_alpha=1, labelsize='15')
plt.axvline(x=10, linestyle="--", color='k',)
plt.axvline(x=60, linestyle="--", color='k',)
plt.tight_layout()
plt.savefig('./Figures/Tg_Strength_init.png', dpi=500)

In [ ]:
# Plot Tg vs. Toughness, comparing Initial vs. Evaluated.

fig, ax = plt.subplots(figsize=(5, 4))
sns.scatterplot(x='Tg', y='Toughness(MJ/m3)', hue='Sample', data=df_nonzero, 
                edgecolor='black', palette=['blue','red'],)

ax.set_ylabel(r'U$_T$ (MJ.m$^{-3}$)', fontsize='15',
              fontname='Arial', fontweight='bold', labelpad=5)
ax.set_xlabel(r'$T_g$ ( °C)', fontsize='18',
              fontname='Arial', fontweight='bold', labelpad=5)
ax.tick_params(direction='out', length=4, width=2, colors='black',
               grid_alpha=1, labelsize='15')
plt.axvline(x=10, linestyle="--", color='k',)
plt.axvline(x=60, linestyle="--", color='k',)
plt.tight_layout()
plt.savefig('./Figures/Tg_Toughness.png', dpi=500)

In [ ]:
# Plot Tg vs. Toughness, Initial

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(5, 4))

# Create a scatterplot comparing Tg and Toughness for the 'Initial' sample
sns.scatterplot(x='Tg', y='Toughness(MJ/m3)', hue='Sample',
                data=df_nonzero.loc[df_nonzero['Sample'] == 'Initial'],
                edgecolor='white', palette=['blue'])

# Set the y-axis label and properties
ax.set_ylabel(r'U$_T$ (MJ.m$^{-3}$)', fontsize=15, fontname='Arial', fontweight='bold', labelpad=5)

# Set the x-axis label and properties
ax.set_xlabel(r'$T_g$ ( °C)', fontsize=18, fontname='Arial', fontweight='bold', labelpad=5)

# Customize tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black', grid_alpha=1, labelsize=15)

# Add vertical dashed lines at x=10 and x=60
plt.axvline(x=10, linestyle="--", color='k')
plt.axvline(x=60, linestyle="--", color='k')

# Ensure a tight layout for the plot
plt.tight_layout()

# Save the plot to an image file
plt.savefig('./Figures/Tg_Toughness_initial.png', dpi=500)


In [ ]:
# Plot Tg vs. Strain (%), comparing Initial vs. Evaluated.

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(5, 4))

# Create a scatterplot comparing Tg and Tensile Strain Percentage for all samples
sns.scatterplot(x='Tg', y='Tensile_Strain_percentage', hue='Sample',
                data=df_nonzero, edgecolor='black', palette=['blue', 'red'])

# Set the y-axis label and properties
ax.set_ylabel('Strain (%)', fontsize=18, fontname='Arial', fontweight='bold', labelpad=5)

# Set the x-axis label and properties
ax.set_xlabel(r'$T_g$ ( °C)', fontsize=18, fontname='Arial', fontweight='bold', labelpad=5)

# Customize tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black', grid_alpha=1, labelsize=15)

# Add vertical dashed lines at x=10 and x=60
plt.axvline(x=10, linestyle="--", color='k')
plt.axvline(x=60, linestyle="--", color='k')

# Ensure a tight layout for the plot
plt.tight_layout()

# Save the plot to an image file
plt.savefig('./Figures/Tg_Strain.png', dpi=500)


In [ ]:
# Histogram of Tg

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(5, 4))

# Set the x-axis label and properties
ax.set_xlabel(r'$T_g$ ( °C)', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)

# Set the y-axis label and properties
ax.set_ylabel('Rel. Abundance', fontsize='18', fontname='Arial', fontweight='bold', labelpad=5)

# Customize tick parameters
ax.tick_params(direction='out', length=4, width=2, colors='black', grid_alpha=1, labelsize=15)

# Define colors for histograms
colors = ['red', 'blue']

# Create histograms for initial and evaluated data
hist_init = plt.hist(df_nonzero['Tg'].loc[df_nonzero['initial'] == 'Initial'], bins=20, color='blue', density=True, alpha=0.7, label='Init.')
hist_eval = plt.hist(df_nonzero['Tg'].loc[df_nonzero['initial'] == 'Eval.'], bins=15, color='red', density=True, alpha=0.7, label='Eval.')

# Add vertical dashed lines at x=9 and x=61
plt.axvline(x=9, linestyle="--", color='k')
plt.axvline(x=61, linestyle="--", color='k')

# Add a legend to distinguish between initial and evaluated data
plt.legend()

# Ensure a tight layout for the plot
plt.tight_layout()

# Save the plot to an image file
plt.savefig('./Figures/Tg_bins.png', dpi=500)


In [ ]:
# Correlation between the Toughness, Strength, and Strain

# Select relevant columns from the DataFrame
df_mechprop = df_nonzero[['Tensile_Strength(MPa)', 'Tensile_Strain_percentage', 'Toughness(MJ/m3)', 'Tg']]

# Rename columns for better readability
df_mechprop = df_mechprop.rename(columns={
    "Tensile_Strength(MPa)": "σ$_{T}$",
    "Tensile_Strain_percentage": "Strain",
    "Toughness(MJ/m3)": "U$_T$",
    "Tg": "$T_g$"
})

# Define a function to calculate Pearson correlation p-values
def pearsonr_pval(x, y):
    return pearsonr(x, y)[1]

# Compute p-values for Pearson correlations
pvalue_pearson = df_mechprop.corr(method=pearsonr_pval)

# Compute Pearson correlations
corr = df_mechprop.corr(method='pearson')

# Create a figure and axis for the plot
f, ax = plt.subplots(figsize=(6, 5))

# Add p-values as text to the plot
for i in range(0, 3):
    plt.text(i + 0.1, i + 0.2, 'Pval: {}'.format(np.round(pvalue_pearson.iloc[i, i + 1], 3)))
for i in range(0, 2):
    plt.text(i + 0.1, i + 1.2, 'Pval: {}'.format(np.round(pvalue_pearson.iloc[i + 2, i], 3)))
for i in range(0, 1):
    plt.text(i + 0.1, i + 2.2, 'Pval: {}'.format(np.round(pvalue_pearson.iloc[i + 3, i], 3)))

# Create a mask to hide lower triangular values
mask = np.invert(np.tril(np.ones_like(corr.iloc[1:4, 0:3], dtype=bool)))

# Define a color map for the heatmap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Create a heatmap of Pearson correlations
sns.heatmap(corr.iloc[1:4, 0:3], annot=True, cmap=cmap, cbar=True, mask=mask, yticklabels='auto', 
            cbar_kws={'label': 'Pearson correlation'})

# Ensure a tight layout for the plot
plt.tight_layout()

# Save the plot to an image file
plt.savefig('./Figures/heatmap_allmechprop.png', dpi=200)


In [ ]:
print ('!!!!!!Done!!!!!!')